In [1]:
# verify that a connection to the metadata store can be established
# note: this notebook should be ran inside of a kubernetes conatiner
from ml_metadata import metadata_store
from ml_metadata.proto import metadata_store_pb2
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.mysql.host = 'mysql'
connection_config.mysql.port = 3306
connection_config.mysql.database = 'mysql'
connection_config.mysql.user = 'root'
connection_config.mysql.password = ''
import sys
print(connection_config)
try:
    store = metadata_store.MetadataStore(connection_config)
except Exception as e:
    print(sys.exc_info()[0], e)

mysql {
  host: "mysql"
  port: 3306
  database: "mysql"
  user: "root"
  password: ""
}

<class 'RuntimeError'> mysql_real_connect failed: errno: 2005, error: Unknown MySQL server host 'mysql' (-2)


In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import datetime
import os
from typing import Any, Iterable, List, Optional, Text, Type

import absl
import apache_beam as beam

from tfx.components.base import base_node
from tfx.orchestration import data_types
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration import tfx_runner
from tfx.orchestration.config import base_component_config
from tfx.orchestration.config import config_utils
from tfx.orchestration.config import pipeline_config
from tfx.orchestration.launcher import base_component_launcher
from tfx.orchestration.launcher import docker_component_launcher
from tfx.orchestration.launcher import kubernetes_component_launcher
from tfx.orchestration.experimental.kubernetes import kubernetes_dag_runner
from tfx.orchestration.test_pipelines.download_grep_print_pipeline import create_pipeline_component_instances
from tfx.utils import telemetry_utils


In [4]:
_tfx_root = os.path.join(os.environ['HOME'], 'tfx')
_pipeline_name = 'test'
#_pipeline_root = os.path.join(_tfx_root, 'pipelines', _pipeline_name)
_pipeline_root = "gs://tfx-eric-default"


def _create_pipeline() -> pipeline.Pipeline:

    pipeline_name = _pipeline_name
    pipeline_root = _pipeline_root

    components = create_pipeline_component_instances("www.google.com", "google")

    # in cluster connection config
    metadata_connection_config = metadata.mysql_metadata_connection_config(
    host='mysql', port=3306, username='root', database='mysql', password='')

    return pipeline.Pipeline(
        pipeline_name=pipeline_name,
        pipeline_root=pipeline_root,
        components=components,
        metadata_connection_config=metadata_connection_config,
        enable_cache=True,
#        beam_pipeline_args=beam_pipeline_args,
    ) 


In [5]:
if __name__ == '__main__':
    # first, create a tfx pipiline
    _pipeline = _create_pipeline()
    # use kubernetes dag runner to run the pipeline
    kubernetes_dag_runner.KubernetesDagRunner().run(tfx_pipeline=_pipeline)

RuntimeError: Failed to establish connection to Metadata storage with error: mysql_real_connect failed: errno: 2005, error: Unknown MySQL server host 'mysql' (-2)